In [ ]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)

mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(max_num_faces = 2)
mpDraw = mp.solutions.drawing_utils

while True:
    succes, img = cap.read()
        
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = faceMesh.process(imgRGB) 
    
    if result.multi_face_landmarks:
        for faceLms in result.multi_face_landmarks:
            # Mostrar todos los puntos
            mpDraw.draw_landmarks(img, faceLms, mpFaceMesh.FACEMESH_CONTOURS)
            
            # Mostrar solo el contorno
            draw_spec = mpDraw.DrawingSpec(thickness=1, circle_radius=0)  
            mpDraw.draw_landmarks(img, faceLms, mpFaceMesh.FACEMESH_CONTOURS, draw_spec, draw_spec)

        
    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == 27:  # Presionar 'Esc' para salir
        break

cap.release()
cv2.destroyAllWindows()

UBICAR UN PUNTO CONCRETO:

In [23]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)

mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(max_num_faces = 2, refine_landmarks=True)
mpDraw = mp.solutions.drawing_utils

while True:
    succes, img = cap.read()
        
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = faceMesh.process(imgRGB) 
    
    if result.multi_face_landmarks:
        for faceLms in result.multi_face_landmarks:
            # Mostrar todos los puntos
            #mpDraw.draw_landmarks(img, faceLms, mpFaceMesh.FACEMESH_CONTOURS)

            # Pintar los puntos del iris
            for id, lm in enumerate(faceLms.landmark): 
                h, w, c = img.shape
                centerX, centerY = int(lm.x*w), int(lm.y*h)
                
                if id in [468, 473]: # Puntos del iris
                    cv2.circle(img, (centerX, centerY), 3, (255,0,255), cv2.FILLED)

        
    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == 27:  # Presionar 'Esc' para salir
        break

cap.release()
cv2.destroyAllWindows()

Seguimiento de retina con el raton

In [34]:
import cv2
import mediapipe as mp
import pyautogui
import numpy as np

cap = cv2.VideoCapture(0)
mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(max_num_faces=1, refine_landmarks=True)
screen_w, screen_h = pyautogui.size()

smooth_factor = 0.2
previous_mouse_x, previous_mouse_y = screen_w // 2, screen_h // 2

# Landmarks clave para el ojo derecho
eye_right_outer = 33
eye_right_inner = 133
iris_center = 468

while True:
    success, img = cap.read()
    if not success:
        continue

    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = faceMesh.process(imgRGB)

    if result.multi_face_landmarks:
        faceLms = result.multi_face_landmarks[0]
        h, w, c = img.shape

        # Obtener coordenadas de ojo y iris
        eye_outer = faceLms.landmark[eye_right_outer]
        eye_inner = faceLms.landmark[eye_right_inner]
        iris = faceLms.landmark[iris_center]

        # Pasar a pixeles
        eye_outer_x, eye_outer_y = int(eye_outer.x * w), int(eye_outer.y * h)
        eye_inner_x, eye_inner_y = int(eye_inner.x * w), int(eye_inner.y * h)
        iris_x, iris_y = int(iris.x * w), int(iris.y * h)

        # Centro del ojo
        eye_center_x = (eye_outer_x + eye_inner_x) // 2
        eye_center_y = (eye_outer_y + eye_inner_y) // 2

        # Dibujo de referencia
        cv2.circle(img, (iris_x, iris_y), 5, (0, 255, 255), cv2.FILLED)
        cv2.circle(img, (eye_center_x, eye_center_y), 3, (0, 0, 255), cv2.FILLED)
        cv2.line(img, (eye_outer_x, eye_outer_y), (eye_inner_x, eye_inner_y), (255, 0, 0), 1)

        # Calcula posición relativa del iris al centro del ojo en un eje normalizado
        eye_width = np.linalg.norm(np.array([eye_outer_x, eye_outer_y]) - np.array([eye_inner_x, eye_inner_y]))
        rel_x = (eye_center_x - iris_x) / eye_width
        rel_y = (eye_center_y - iris_y) / eye_width


        # Ajustar sensibilidad
        amplification_factor = 2.5  # Puedes ajustar este valor para hacerlo más o menos sensible

        mapped_x = screen_w / 2 + rel_x * screen_w * amplification_factor
        mapped_y = screen_h / 2 + rel_y * screen_h * amplification_factor

        # Suavizado
        smoothed_x = previous_mouse_x + (mapped_x - previous_mouse_x) * smooth_factor
        smoothed_y = previous_mouse_y + (mapped_y - previous_mouse_y) * smooth_factor

        pyautogui.moveTo(smoothed_x, smoothed_y)
        previous_mouse_x, previous_mouse_y = smoothed_x, smoothed_y

    cv2.imshow("Eye Tracking Localized - Right Iris (468)", img)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
